---

In [ ]:
"""
Radiomics Feature Extraction Pipeline for Head & Neck Cancer CT Imaging

This module implements a comprehensive radiomics feature extraction workflow
for analyzing CT imaging data with RT structure sets (RTSTRUCT). The pipeline
extracts quantitative imaging features following Image Biomarker Standardization
Initiative (IBSI) guidelines using the PyRadiomics framework.

Technical Environment:
    Python: 3.8.17
    Environment: PyRadiomics Conda Environment
    
Author: Hasan Shaikh
Contact: hasanshaikh3198@gmail.com
Repository: https://github.com/hash123shaikh

References:
    - Van Griethuysen et al. (2017). Computational Radiomics System to Decode 
      the Radiographic Phenotype. Cancer Research.
    - Zwanenburg et al. (2020). The Image Biomarker Standardization Initiative.
      Radiology.

License: MIT License
Version: 1.0.0
Last Updated: 2025
"""

---

### **Cell 1: Dependency Installation**

In [ ]:
"""
Package Installation

Installs required dependencies for medical image processing and radiomics
feature extraction. Execute this cell only if packages are not already
installed in your Python environment.

Dependencies:
    - pyradiomics   : Feature extraction framework (IBSI-compliant)
    - rt-utils      : RT structure set manipulation and conversion
    - SimpleITK     : Medical image I/O and processing
    - pydicom       : DICOM standard implementation
    - pandas        : Tabular data management
    - numpy         : Numerical computing
"""

In [ ]:
!pip install pyradiomics
!pip install rt-utils
!pip install SimpleITK
!pip install pydicom
!pip install pandas
!pip install numpy

---

### **Cell 2: Library Imports and Environment Configuration**

In [ ]:
"""
Core Library Imports

Imports essential libraries for medical image processing, DICOM manipulation,
and radiomics feature extraction. Each library serves a specific role in the
computational pipeline.

Library Roles:
    - SimpleITK    : Medical image I/O, resampling, and spatial operations
    - pydicom      : DICOM file parsing and metadata extraction
    - radiomics    : IBSI-standardized feature extraction
    - pandas       : Feature matrix construction and CSV export
    - numpy        : Array operations and numerical computing
    - rt_utils     : RT structure set conversion to binary masks
    - pathlib      : Cross-platform file path handling
"""

In [79]:
import os
import SimpleITK as sitk
import pydicom
from radiomics import featureextractor
import pandas as pd
import numpy as np
from pathlib import Path
import rt_utils

print("Status: All required libraries imported successfully")


Status: All required libraries imported successfully


---

### **Cell 3: Working Directory Configuration**

In [ ]:
"""
Dataset Path Configuration

Establishes the root directory containing patient imaging data. The dataset
should follow a standardized directory structure with separate folders for
each patient containing CT and RTSTRUCT subdirectories.

Expected Directory Structure:
    base_folder/
    ├── PATIENT-001/
    │   ├── CT/              # CT DICOM series
    │   └── RTSTRUCT/        # RT structure set DICOM
    ├── PATIENT-002/
    └── ...

Configuration Notes:
    - Use absolute paths for production environments
    - Ensure read permissions for all subdirectories
    - Verify path exists before proceeding with batch processing
"""

In [80]:
# CONFIGURATION: Update this path to match your dataset location
base_folder = "/home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images"

# Set working directory
os.chdir(base_folder)
print(f"Working directory configured: {os.getcwd()}")

# Validate path exists
if not os.path.exists(base_folder):
    raise FileNotFoundError(f"Dataset path not found: {base_folder}")

Working directory configured: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images


---

### **Cell 4: Pipeline Configuration Parameters**

In [ ]:
"""
Analysis Configuration

Defines key parameters for the radiomics extraction pipeline including
target structure identification, output specifications, and feature extraction
mode.

Configuration Parameters:
    TARGET_ROI: Structure name for feature extraction
        - Common values: "GTV-1" (primary tumor), "GTV", "CTV", "PTV"
        - Must match nomenclature in RTSTRUCT files
        - Case-sensitive string matching
        
    OUTPUT_CSV: Filename for feature matrix export
        - CSV format with patients as rows, features as columns
        - Includes metadata columns (PatientID, ROI_Name)
        
    FEATURE_MODE: Feature extraction complexity
        - "original": First-order, shape, and texture features only
        - "wavelet": Includes wavelet decomposition features
        - "log": Includes Laplacian of Gaussian filtered features
        - "all": Comprehensive feature extraction (all image types)

Computational Considerations:
    - "original" mode: ~100 features, fastest execution
    - "wavelet" mode: ~1000 features, 5-10x slower
    - "all" mode: ~2000 features, 10-20x slower
"""

In [81]:
# CONFIGURATION: Adjust these parameters for your analysis
TARGET_ROI = "GTV-1"
OUTPUT_CSV = "radiomics_features.csv"
FEATURE_MODE = "original"  # Options: "original", "wavelet", "log", "all"

# Display current configuration
print("="*70)
print("PIPELINE CONFIGURATION")
print("="*70)
print(f"Target Structure: {TARGET_ROI}")
print(f"Output File: {OUTPUT_CSV}")
print(f"Feature Extraction Mode: {FEATURE_MODE}")
print(f"Dataset Location: {base_folder}")
print("="*70)

# Validate configuration
if FEATURE_MODE not in ["original", "wavelet", "log", "all"]:
    raise ValueError(f"Invalid FEATURE_MODE: {FEATURE_MODE}")

PIPELINE CONFIGURATION
Target Structure: GTV-1
Output File: radiomics_features.csv
Feature Extraction Mode: original
Dataset Location: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images


---

### **Cell 5: CT Series Loading Function**

In [ ]:
"""
DICOM CT Series Reader

Implements robust loading of CT imaging series from DICOM files using
SimpleITK's GDCM-based series reader. Handles multi-slice CT volumes with
proper spatial metadata preservation.

Function: load_ct_series()

Technical Details:
    - Uses GDCM (Grassroots DICOM) library for DICOM parsing
    - Automatically sorts slices by ImagePositionPatient
    - Preserves spatial metadata (spacing, origin, direction cosines)
    - Returns 3D volume as SimpleITK Image object
    
Error Handling:
    - Raises exception if no DICOM files found
    - Validates series integrity (consistent dimensions, spacing)
    - Handles mixed-series folders by selecting first valid series
"""

In [82]:
def load_ct_series(ct_folder: str) -> sitk.Image:
    """
    Load CT DICOM series and construct 3D volume.
    
    Reads all DICOM files in the specified directory, identifies series
    using GDCM metadata, and constructs a 3D SimpleITK Image with proper
    spatial information.
    
    Parameters
    ----------
    ct_folder : str
        Path to directory containing CT DICOM files (.dcm)
        
    Returns
    -------
    sitk.Image
        3D CT volume with spatial metadata (spacing, origin, direction)
        Intensity values in Hounsfield Units (HU)
        
    Raises
    ------
    RuntimeError
        If no DICOM series found in specified directory
    IOError
        If DICOM files are corrupted or unreadable
        
    Notes
    -----
    - Automatically handles slice ordering using ImagePositionPatient
    - Assumes single CT series per directory
    - Supports standard DICOM transfer syntaxes
    
    Examples
    --------
    >>> ct_image = load_ct_series("/path/to/ct/folder")
    >>> print(f"Image size: {ct_image.GetSize()}")
    Image size: (512, 512, 120)
    """
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(ct_folder)
    
    if not dicom_names:
        raise RuntimeError(f"No DICOM series found in {ct_folder}")
    
    reader.SetFileNames(dicom_names)
    ct_image = reader.Execute()
    
    return ct_image

print("Status: Function 'load_ct_series' defined successfully")

Status: Function 'load_ct_series' defined successfully


---

### **Cell 6: RT Structure Search Function**

In [ ]:
"""
RT Structure Set ROI Locator

Implements intelligent search algorithm to locate target ROI across multiple
RTSTRUCT DICOM files. Handles datasets where multiple RTSTRUCT files may
exist per patient (e.g., different treatment planning stages).

Function: find_roi_in_rtstructs()

Search Algorithm:
    1. Enumerate all DICOM files in RTSTRUCT directory
    2. Sequentially load each RTSTRUCT file
    3. Extract ROI name list from structure set metadata
    4. Perform case-sensitive string matching against target
    5. Return first RTSTRUCT containing target ROI
    
Common ROI Nomenclature Patterns:
    - Primary tumor    : GTV, GTV-1, GTV-Primary, GTV_1
    - Clinical target  : CTV, CTV-High, CTV-Low
    - Planning target  : PTV, PTV-1, PTV54, PTV60
    - Organs at risk   : Lung_L, Lung_R, Heart, Esophagus
"""

In [83]:
def find_roi_in_rtstructs(
    rtstruct_folder: str,
    ct_folder: str,
    target_roi_name: str = "GTV-1"
) -> tuple:
    """
    Locate RTSTRUCT file containing specified ROI.
    
    Searches through all RTSTRUCT DICOM files in directory to find
    the structure set containing the target ROI. Required for datasets
    with multiple RTSTRUCT files per patient.
    
    Parameters
    ----------
    rtstruct_folder : str
        Path to directory containing RTSTRUCT DICOM files
    ct_folder : str
        Path to CT series directory (required for spatial registration)
    target_roi_name : str, default="GTV-1"
        Exact name of target ROI structure
        
    Returns
    -------
    tuple[RTStruct, str]
        - RTStruct object from rt_utils library
        - Confirmed ROI name (for validation)
        
    Raises
    ------
    FileNotFoundError
        If no RTSTRUCT files exist in specified directory
    ValueError
        If target ROI not found in any RTSTRUCT file
    RuntimeError
        If RTSTRUCT-CT spatial registration fails
        
    Notes
    -----
    - ROI name matching is case-sensitive
    - Returns immediately upon finding first match
    - Logs all available ROI names for debugging
    - Requires CT series for coordinate system registration
    
    Examples
    --------
    >>> rt_struct, roi_name = find_roi_in_rtstructs(
    ...     "/path/to/rtstruct",
    ...     "/path/to/ct",
    ...     "GTV-1"
    ... )
    Found 2 RTSTRUCT file(s)
      Checking: RS.1.2.3.dcm
        Available ROIs: ['GTV-1', 'CTV', 'PTV', 'Lung_L', 'Lung_R']
        ✓ Found 'GTV-1' in RS.1.2.3.dcm
    """
    rtstruct_files = list(Path(rtstruct_folder).glob("*.dcm"))
    
    if not rtstruct_files:
        raise FileNotFoundError(
            f"No RTSTRUCT DICOM files found in {rtstruct_folder}"
        )
    
    print(f"Found {len(rtstruct_files)} RTSTRUCT file(s) to search")
    
    # Search through each RTSTRUCT file
    for rtstruct_file in rtstruct_files:
        try:
            print(f"  Examining: {rtstruct_file.name}")
            
            # Load RTSTRUCT with spatial registration to CT
            rt_struct = rt_utils.RTStructBuilder.create_from(
                dicom_series_path=str(ct_folder),
                rt_struct_path=str(rtstruct_file)
            )
            
            # Extract ROI nomenclature
            roi_names = rt_struct.get_roi_names()
            print(f"    Available structures: {roi_names}")
            
            # Check for target ROI presence
            if target_roi_name in roi_names:
                print(f"    Status: Target structure '{target_roi_name}' located")
                return rt_struct, target_roi_name
                
        except Exception as e:
            print(f"    Error processing {rtstruct_file.name}: {str(e)[:100]}")
            continue
    
    # Target ROI not found in any file
    raise ValueError(
        f"Structure '{target_roi_name}' not found in any RTSTRUCT file. "
        f"Verify structure name matches dataset nomenclature."
    )

print("Status: Function 'find_roi_in_rtstructs' defined successfully")

Status: Function 'find_roi_in_rtstructs' defined successfully


---

### **Cell 7: RTSTRUCT to Mask Conversion Function**

In [ ]:
"""
Binary Mask Generation from RT Structure Sets

Converts vector-based RTSTRUCT contours to binary voxel masks compatible
with CT image space. Implements spatial registration and dimension matching
to ensure proper alignment for radiomics feature extraction.

Function: load_rtstruct_as_mask()

Technical Process:
    1. Extract contour coordinates from RTSTRUCT
    2. Rasterize contours to binary mask array
    3. Handle axis ordering differences (rt_utils vs SimpleITK)
    4. Copy spatial metadata from reference CT image
    5. Validate dimensional consistency
    
Coordinate System Handling:
    - DICOM RTSTRUCT: Patient-based coordinate system (mm)
    - rt_utils output: (Z, Y, X) array ordering
    - SimpleITK Image: (X, Y, Z) memory layout
    - Requires careful transpose operations for alignment
"""

In [84]:
def load_rtstruct_as_mask(
    rt_struct,
    ct_image: sitk.Image,
    roi_name: str
) -> sitk.Image:
    """
    Convert RTSTRUCT contour to binary segmentation mask.
    
    Rasterizes vector-based structure contours to voxel-based binary mask
    with spatial alignment to CT image. Ensures proper dimension matching
    and coordinate system consistency.
    
    Parameters
    ----------
    rt_struct : RTStruct
        RT structure set object from rt_utils
    ct_image : sitk.Image
        Reference CT image for spatial parameters
    roi_name : str
        Name of ROI structure to convert
        
    Returns
    -------
    sitk.Image
        Binary mask (0=background, 1=ROI)
        Same dimensions and spatial metadata as ct_image
        Data type: unsigned 8-bit integer
        
    Raises
    ------
    ValueError
        If mask dimensions don't match CT image after alignment
    KeyError
        If specified ROI name not found in structure set
        
    Notes
    -----
    - Handles axis ordering: rt_utils (Z,Y,X) → SimpleITK (X,Y,Z)
    - Copies spacing, origin, direction from reference CT
    - Binary mask: 1=inside ROI, 0=outside
    - Suitable for PyRadiomics feature extraction
    
    Technical Details
    -----------------
    Coordinate System Transformations:
        1. RTSTRUCT contours (mm) → rt_utils rasterization
        2. NumPy array (Z,Y,X) → Transpose if needed
        3. SimpleITK Image (X,Y,Z) → Copy CT metadata
        
    Examples
    --------
    >>> mask = load_rtstruct_as_mask(rt_struct, ct_image, "GTV-1")
    Extracting mask for ROI: GTV-1
      Original mask shape: (120, 512, 512)
      CT image size: (512, 512, 120)
      Final mask size: (512, 512, 120)
    >>> print(f"ROI voxel count: {sitk.GetArrayFromImage(mask).sum()}")
    ROI voxel count: 15847
    """
    print(f"Converting structure to binary mask: {roi_name}")
    
    # Step 1: Rasterize RTSTRUCT contours to NumPy array
    mask_array = rt_struct.get_roi_mask_by_name(roi_name)
    
    print(f"  Raw mask dimensions: {mask_array.shape}")
    print(f"  Reference CT dimensions: {ct_image.GetSize()}")
    
    # Step 2: Handle axis ordering discrepancy
    # rt_utils returns (Z, Y, X), but SimpleITK expects array in format
    # compatible with GetImageFromArray which reads as (Z, Y, X) → (X, Y, Z) image
    # If dimensions don't match, transpose is required
    
    if mask_array.shape != ct_image.GetSize()[::-1]:
        # Transpose from (Z, Y, X) to match CT array ordering
        mask_array = np.transpose(mask_array, (2, 1, 0))
        print(f"  Transposed mask dimensions: {mask_array.shape}")
    
    # Step 3: Convert to SimpleITK Image format
    mask_image = sitk.GetImageFromArray(mask_array.astype(np.uint8))
    
    # Step 4: Copy spatial metadata from reference CT
    # Essential for proper spatial alignment in radiomics extraction
    mask_image.SetSpacing(ct_image.GetSpacing())
    mask_image.SetOrigin(ct_image.GetOrigin())
    mask_image.SetDirection(ct_image.GetDirection())
    
    # Step 5: Validate dimensional consistency
    if mask_image.GetSize() != ct_image.GetSize():
        raise ValueError(
            f"Dimension mismatch after conversion: "
            f"mask={mask_image.GetSize()}, CT={ct_image.GetSize()}"
        )
    
    print(f"  Final mask dimensions: {mask_image.GetSize()}")
    
    # Calculate ROI statistics
    mask_array_final = sitk.GetArrayFromImage(mask_image)
    voxel_count = np.sum(mask_array_final)
    volume_cc = voxel_count * np.prod(ct_image.GetSpacing()) / 1000  # mm³ to cc
    
    print(f"  ROI voxel count: {voxel_count}")
    print(f"  ROI volume: {volume_cc:.2f} cc")
    
    return mask_image

print("Status: Function 'load_rtstruct_as_mask' defined successfully")


Status: Function 'load_rtstruct_as_mask' defined successfully


---

### **Cell 8: Radiomics Feature Extraction Function**

In [ ]:
"""
PyRadiomics Feature Extraction Interface

Implements comprehensive radiomics feature extraction following IBSI guidelines.
Extracts quantitative imaging biomarkers across multiple feature classes and
optional image preprocessing transformations.

Function: extract_radiomics_features()

Feature Classes Extracted:
    1. Shape Features (14 features)
       - Volume, surface area, sphericity, compactness
       - Major/minor/least axis lengths
       - Elongation, flatness, spherical disproportion
       
    2. First-Order Features (18 features)
       - Statistical moments: mean, median, variance, skewness, kurtosis
       - Intensity metrics: min, max, range, energy
       - Histogram features: entropy, uniformity, percentiles
       
    3. Texture Features (~75 features per image type)
       - GLCM: Gray Level Co-occurrence Matrix (24 features)
       - GLRLM: Gray Level Run Length Matrix (16 features)
       - GLSZM: Gray Level Size Zone Matrix (16 features)
       - GLDM: Gray Level Dependence Matrix (14 features)
       - NGTDM: Neighboring Gray Tone Difference Matrix (5 features)

Image Preprocessing Options:
    - Original: Raw CT intensities (Hounsfield Units)
    - Wavelet: 8 wavelet decompositions (LLL, LLH, LHL, etc.)
    - LoG: Laplacian of Gaussian filtering (multiple sigma values)
    - Square: Squared intensities
    - Square Root: Square root transformation
    - Logarithm: Logarithmic transformation
    - Exponential: Exponential transformation

Computational Complexity:
    - Original: ~107 features, ~5 seconds per patient
    - Wavelet: ~856 features, ~40 seconds per patient
    - All: ~2000+ features, ~120 seconds per patient
"""

In [85]:
def extract_radiomics_features(
    ct_image: sitk.Image,
    mask_image: sitk.Image,
    patient_id: str,
    roi_name: str,
    feature_mode: str = "original"
) -> dict:
    """
    Extract IBSI-compliant radiomics features from CT and mask.
    
    Computes quantitative imaging biomarkers using PyRadiomics framework.
    Features include shape, first-order statistics, and texture metrics
    across multiple gray-level matrices.
    
    Parameters
    ----------
    ct_image : sitk.Image
        CT volume in Hounsfield Units
    mask_image : sitk.Image
        Binary ROI mask (0=background, 1=ROI)
    patient_id : str
        Patient identifier for result tracking
    roi_name : str
        ROI structure name
    feature_mode : str, default="original"
        Feature extraction complexity:
        - "original": Base features only (~107 features)
        - "wavelet": Includes wavelet transforms (~856 features)
        - "log": Includes LoG filtering (~428 features)
        - "all": All transformations (~2000+ features)
        
    Returns
    -------
    dict
        Feature dictionary with keys:
        - 'PatientID': Patient identifier
        - 'ROI_Name': Structure name
        - 'original_shape_*': Shape features
        - 'original_firstorder_*': First-order features
        - 'original_glcm_*': GLCM texture features
        - 'original_glrlm_*': GLRLM texture features
        - 'original_glszm_*': GLSZM texture features
        - 'original_gldm_*': GLDM texture features
        - 'original_ngtdm_*': NGTDM texture features
        - (Additional prefixes for wavelet/log if enabled)
        
    Raises
    ------
    RuntimeError
        If PyRadiomics feature extraction fails
    ValueError
        If ct_image and mask_image have mismatched dimensions
        
    Notes
    -----
    - All features follow IBSI nomenclature
    - Diagnostic information excluded from output
    - Wavelet decomposition: 8 subbands per texture class
    - LoG filtering: Multiple sigma values for scale analysis
    
    Performance Considerations
    --------------------------
    - Original mode: Suitable for large cohorts (>1000 patients)
    - Wavelet mode: Balance between depth and speed
    - All mode: Comprehensive but computationally expensive
    
    References
    ----------
    Van Griethuysen et al. (2017). Computational Radiomics System to Decode
    the Radiographic Phenotype. Cancer Research, 77(21), e104-e107.
    
    Examples
    --------
    >>> features = extract_radiomics_features(
    ...     ct_image, mask_image, "PATIENT-001", "GTV-1", "original"
    ... )
    Extracting radiomics features...
      Using original image only...
      Feature extraction complete: 107 features
    >>> print(f"Mean HU: {features['original_firstorder_Mean']:.2f}")
    Mean HU: -147.32
    """
    # Initialize PyRadiomics feature extractor
    extractor = featureextractor.RadiomicsFeatureExtractor()
    
    # Configure image preprocessing based on feature mode
    if feature_mode == "wavelet":
        print("  Enabling wavelet decomposition transforms...")
        extractor.enableImageTypeByName('Wavelet')
        print("    8 wavelet subbands will be analyzed")
        
    elif feature_mode == "log":
        print("  Enabling Laplacian of Gaussian filtering...")
        extractor.enableImageTypeByName('LoG')
        print("    Multiple sigma values for scale-space analysis")
        
    elif feature_mode == "all":
        print("  Enabling comprehensive image preprocessing...")
        extractor.enableAllImageTypes()
        print("    Includes: Original, Wavelet, LoG, Square, SquareRoot, etc.")
        
    else:  # "original"
        print("  Using original image intensity values only...")
        print("    No preprocessing transformations applied")
    
    # Execute feature extraction
    print("  Computing radiomics features...")
    try:
        features = extractor.execute(ct_image, mask_image)
    except Exception as e:
        raise RuntimeError(f"Feature extraction failed: {str(e)}")
    
    # Construct output dictionary with metadata
    feature_dict = {
        'PatientID': patient_id,
        'ROI_Name': roi_name
    }
    
    # Filter and add extracted features (exclude diagnostics)
    feature_count = 0
    for key, value in features.items():
        if not key.startswith('diagnostics_'):
            feature_dict[key] = value
            feature_count += 1
    
    print(f"  Feature extraction complete: {feature_count} features computed")
    
    return feature_dict

print("Status: Function 'extract_radiomics_features' defined successfully")


Status: Function 'extract_radiomics_features' defined successfully


---

### **Cell 9: Single Patient Processing Function**

In [ ]:
"""
Individual Patient Processing Workflow

Orchestrates complete radiomics extraction pipeline for a single patient.
Integrates CT loading, RTSTRUCT processing, mask generation, and feature
extraction into a unified workflow with comprehensive error handling.

Function: process_patient_folder()

Pipeline Stages:
    1. Directory validation (CT and RTSTRUCT folders)
    2. CT DICOM series loading
    3. RTSTRUCT search for target ROI
    4. Binary mask generation
    5. Radiomics feature extraction
    6. Error handling and logging

Expected Directory Structure:
    PatientFolder/
    ├── CT/              # or "CT SCAN", configurable
    │   ├── slice001.dcm
    │   ├── slice002.dcm
    │   └── ...
    └── RTSTRUCT/
        ├── RS.dcm
        └── ...

Failure Modes and Handling:
    - Missing directories: Returns None, logs warning
    - No DICOM files: Returns None, logs error
    - ROI not found: Raises ValueError with available ROIs
    - Mask generation failure: Propagates exception
    - Feature extraction failure: Propagates exception
"""

In [86]:
def process_patient_folder(
    patient_folder: str,
    target_roi: str = "GTV-1",
    feature_mode: str = "original"
) -> dict:
    """
    Execute complete radiomics pipeline for single patient.
    
    Orchestrates CT loading, structure processing, and feature extraction
    with comprehensive error handling and progress logging.
    
    Parameters
    ----------
    patient_folder : str
        Path to patient directory containing CT and RTSTRUCT folders
    target_roi : str, default="GTV-1"
        Target structure name for analysis
    feature_mode : str, default="original"
        Feature extraction mode: "original", "wavelet", "log", or "all"
        
    Returns
    -------
    dict or None
        Dictionary of extracted features if successful, None if processing failed
        Includes PatientID, ROI_Name, and all computed features
        
    Raises
    ------
    ValueError
        If target ROI not found in any RTSTRUCT file
    RuntimeError
        If feature extraction fails
        
    Notes
    -----
    - Patient ID derived from folder name
    - Logs detailed progress for pipeline monitoring
    - Non-critical failures return None (missing folders)
    - Critical failures raise exceptions (ROI not found)
    
    Directory Naming:
    - Modify ct_folder variable if your CT folder has different name
    - Common alternatives: "CT", "CT SCAN", "CT_SCAN", "IMAGES"
    
    Examples
    --------
    >>> features = process_patient_folder(
    ...     "/data/LUNG1-001",
    ...     target_roi="GTV-1",
    ...     feature_mode="original"
    ... )
    ============================================================
    Processing patient: LUNG1-001
    ============================================================
    Loading CT series...
      CT Image size: (512, 512, 120)
    Searching for ROI 'GTV-1'...
    Found 1 RTSTRUCT file(s) to search
      Examining: RS.1.2.3.dcm
        Available structures: ['GTV-1', 'CTV', 'PTV']
        Status: Target structure 'GTV-1' located
    Converting structure to binary mask: GTV-1
      ...
    ✓ Successfully extracted 107 features
    """
    patient_id = Path(patient_folder).name
    
    print(f"\n{'='*70}")
    print(f"Processing Patient: {patient_id}")
    print(f"{'='*70}")
    
    # Define folder paths (modify if your naming convention differs)
    ct_folder = Path(patient_folder) / "CT"
    rtstruct_folder = Path(patient_folder) / "RTSTRUCT"
    
    # Stage 1: Validate directory structure
    if not ct_folder.exists():
        print(f"Error: CT directory not found for patient {patient_id}")
        print(f"  Expected location: {ct_folder}")
        return None
    
    if not rtstruct_folder.exists():
        print(f"Error: RTSTRUCT directory not found for patient {patient_id}")
        print(f"  Expected location: {rtstruct_folder}")
        return None
    
    try:
        # Stage 2: Load CT imaging series
        print("Stage 1/4: Loading CT DICOM series...")
        ct_image = load_ct_series(str(ct_folder))
        print(f"  CT volume dimensions: {ct_image.GetSize()}")
        print(f"  Voxel spacing: {ct_image.GetSpacing()} mm")
        
        # Stage 3: Locate target ROI in RTSTRUCT files
        print(f"Stage 2/4: Locating ROI '{target_roi}'...")
        rt_struct, roi_name = find_roi_in_rtstructs(
            str(rtstruct_folder),
            str(ct_folder),
            target_roi
        )
        
        # Stage 4: Convert RTSTRUCT to binary mask
        print("Stage 3/4: Generating binary segmentation mask...")
        mask_image = load_rtstruct_as_mask(rt_struct, ct_image, roi_name)
        print("  Mask generation successful")
        
        # Stage 5: Extract radiomics features
        print("Stage 4/4: Extracting radiomics features...")
        features = extract_radiomics_features(
            ct_image, mask_image, patient_id, roi_name, feature_mode
        )
        
        print(f"Status: Patient {patient_id} processed successfully")
        print(f"  Total features extracted: {len(features) - 2}")
        
        return features
        
    except Exception as e:
        print(f"Error: Processing failed for patient {patient_id}")
        print(f"  Exception: {str(e)}")
        return None

print("Status: Function 'process_patient_folder' defined successfully")


Status: Function 'process_patient_folder' defined successfully


---

### **Cell 10: Batch Processing Function**

In [ ]:
"""
Cohort-Level Batch Processing Pipeline

Implements automated batch processing for large patient cohorts with
progress tracking, error handling, and CSV export functionality.

Function: process_multiple_patients()

Workflow:
    1. Enumerate all patient directories
    2. Sequential processing with progress indicators
    3. Feature aggregation across cohort
    4. Failure tracking and reporting
    5. CSV export of complete feature matrix

Output Format:
    CSV file with structure:
    - Rows: Individual patients
    - Columns: PatientID | ROI_Name | Feature_1 | Feature_2 | ...
    - Values: Numerical feature values

Performance Considerations:
    - Sequential processing (no parallelization)
    - Memory usage: O(n_patients × n_features)
    - Typical throughput: 10-20 patients/minute (original mode)
    
Quality Control:
    - Logs all failures with patient IDs
    - Continues processing despite individual failures
    - Final summary statistics
"""

In [87]:
def process_multiple_patients(
    base_folder: str,
    output_csv: str = "radiomics_features.csv",
    target_roi: str = "GTV-1",
    feature_mode: str = "original"
) -> pd.DataFrame:
    """
    Execute batch radiomics extraction across patient cohort.
    
    Processes all patient directories in base folder, extracts features,
    and exports results to CSV format suitable for machine learning pipelines.
    
    Parameters
    ----------
    base_folder : str
        Root directory containing all patient folders
    output_csv : str, default="radiomics_features.csv"
        Output filename for feature matrix
    target_roi : str, default="GTV-1"
        Target structure name for analysis
    feature_mode : str, default="original"
        Feature extraction mode
        
    Returns
    -------
    pd.DataFrame or None
        Feature matrix with patients as rows, features as columns
        Returns None if no patients processed successfully
        
    Notes
    -----
    - Processing continues despite individual patient failures
    - Failed patients logged for manual review
    - CSV includes PatientID and ROI_Name columns
    - Feature columns named following PyRadiomics convention
    
    Quality Metrics:
    - Success rate = successful / total patients
    - Acceptable success rate: >90%
    - Review failures if rate <80%
    
    Examples
    --------
    >>> df = process_multiple_patients(
    ...     base_folder="/data/MAASTRO",
    ...     output_csv="lung_features.csv",
    ...     target_roi="GTV-1",
    ...     feature_mode="original"
    ... )
    ============================================================
    BATCH PROCESSING: 422 patients
    Target ROI: GTV-1
    Mode: original
    ============================================================
    [1/422] Processing Patient: LUNG1-001
    ...
    ✓ Successfully processed: 398 patients
    ✓ Features saved to: lung_features.csv
    ✓ Total features per patient: 107
    """
    all_features = []
    failed_patients = []
    
    # Enumerate patient directories
    patient_folders = sorted([
        f for f in Path(base_folder).iterdir() 
        if f.is_dir()
    ])
    
    print(f"\n{'='*70}")
    print(f"BATCH PROCESSING INITIATED")
    print(f"{'='*70}")
    print(f"Total patients: {len(patient_folders)}")
    print(f"Target ROI: {target_roi}")
    print(f"Feature mode: {feature_mode}")
    print(f"Output file: {output_csv}")
    print(f"{'='*70}\n")
    
    # Sequential patient processing
    for idx, patient_folder in enumerate(patient_folders, 1):
        print(f"\n[{idx}/{len(patient_folders)}] ", end="")
        
        try:
            features = process_patient_folder(
                str(patient_folder),
                target_roi,
                feature_mode
            )
            
            if features is not None:
                all_features.append(features)
            else:
                failed_patients.append(patient_folder.name)
                
        except Exception as e:
            print(f"Critical error for {patient_folder.name}: {str(e)[:100]}")
            failed_patients.append(patient_folder.name)
            continue
    
    # Generate summary and export results
    print(f"\n{'='*70}")
    print("BATCH PROCESSING SUMMARY")
    print(f"{'='*70}")
    
    if all_features:
        # Convert to DataFrame
        df = pd.DataFrame(all_features)
        
        # Export to CSV
        df.to_csv(output_csv, index=False)
        
        # Calculate statistics
        success_rate = len(all_features) / len(patient_folders) * 100
        
        print(f"Successfully processed: {len(all_features)} patients ({success_rate:.1f}%)")
        print(f"Failed processing: {len(failed_patients)} patients")
        print(f"Features per patient: {len(df.columns) - 2}")
        print(f"Output file: {output_csv}")
        print(f"File size: {os.path.getsize(output_csv) / 1024 / 1024:.2f} MB")
        
        # List failed patients
        if failed_patients:
            print(f"\nFailed Patients (n={len(failed_patients)}):")
            for patient in failed_patients[:20]:  # Show first 20
                print(f"  - {patient}")
            if len(failed_patients) > 20:
                print(f"  ... and {len(failed_patients) - 20} more")
        
        print(f"\nQuality Assessment:")
        if success_rate >= 90:
            print(f"  Status: Excellent ({success_rate:.1f}% success rate)")
        elif success_rate >= 80:
            print(f"  Status: Good ({success_rate:.1f}% success rate)")
        elif success_rate >= 70:
            print(f"  Status: Acceptable ({success_rate:.1f}% success rate)")
            print(f"  Recommendation: Review failed cases")
        else:
            print(f"  Status: Poor ({success_rate:.1f}% success rate)")
            print(f"  Recommendation: Investigate data quality issues")
        
        return df
        
    else:
        print("Error: No patients processed successfully")
        print(f"Total failures: {len(failed_patients)}")
        
        if failed_patients:
            print("\nFailed patients:")
            for patient in failed_patients:
                print(f"  - {patient}")
        
        return None

print("Status: Function 'process_multiple_patients' defined successfully")

Status: Function 'process_multiple_patients' defined successfully


---

### **Cell 11: Single Patient Testing (Optional)**

In [ ]:
"""
Individual Patient Testing Protocol

Provides isolated testing capability for single patient processing before
executing batch analysis on complete cohort. Useful for:
    - Pipeline validation
    - Parameter optimization
    - Debugging data-specific issues
    - Feature preview

Usage:
    1. Uncomment code block below
    2. Set test_patient to actual patient folder name
    3. Execute cell to process single patient
    4. Review output CSV for feature inspection
"""

In [ ]:
# TESTING: Uncomment to test single patient processing
# test_patient = "LUNG1-001"  # Replace with actual patient ID
# 
# print("="*70)
# print("SINGLE PATIENT TEST MODE")
# print("="*70)
# print(f"Test patient: {test_patient}")
# print(f"Target ROI: {TARGET_ROI}")
# print(f"Feature mode: {FEATURE_MODE}")
# print("="*70 + "\n")
# 
# features = process_patient_folder(
#     test_patient,
#     target_roi=TARGET_ROI,
#     feature_mode=FEATURE_MODE
# )
# 
# if features:
#     # Export test results
#     df_test = pd.DataFrame([features])
#     test_output = f"{test_patient}_radiomics_test.csv"
#     df_test.to_csv(test_output, index=False)
#     
#     print(f"\n{'='*70}")
#     print("TEST SUCCESSFUL")
#     print(f"{'='*70}")
#     print(f"Features extracted: {len(features) - 2}")
#     print(f"Output file: {test_output}")
#     
#     # Display sample features
#     print("\nSample Features (first 10):")
#     for i, (key, value) in enumerate(list(features.items())[:10], 1):
#         if isinstance(value, float):
#             print(f"  {i}. {key}: {value:.4f}")
#         else:
#             print(f"  {i}. {key}: {value}")
#     
#     print("\nRecommendation: Review output CSV before batch processing")
# else:
#     print("\nTest failed - review error messages above")

---

### **Cell 12: Configuration Verification**

In [ ]:
"""
Pre-Execution Configuration Review

Final verification step before batch processing. Displays all configuration
parameters for user confirmation.

Checklist:
    ✓ Base folder path correct and accessible
    ✓ Target ROI name matches dataset nomenclature
    ✓ Feature mode appropriate for analysis goals
    ✓ Output filename unique (won't overwrite existing results)
"""

In [88]:
print("="*70)
print("CONFIGURATION VERIFICATION")
print("="*70)
print("\nReview settings before executing batch processing:\n")
print(f"  Dataset location: {base_folder}")
print(f"  Target structure: {TARGET_ROI}")
print(f"  Feature mode: {FEATURE_MODE}")
print(f"  Output file: {OUTPUT_CSV}")
print(f"  Patient folders detected: {len(list(Path(base_folder).iterdir()))}")
print("\n" + "="*70)
print("If configuration correct, proceed to next cell")
print("Otherwise, return to Cell 4 to modify parameters")
print("="*70)

CONFIGURATION VERIFICATION

Review settings before executing batch processing:

  Dataset location: /home/radiomicsserver/Downloads/hasan/MAASTRO Dataset/Images
  Target structure: GTV-1
  Feature mode: original
  Output file: radiomics_features.csv
  Patient folders detected: 137

If configuration correct, proceed to next cell
Otherwise, return to Cell 4 to modify parameters


---

### **Cell 13: Execute Batch Processing**

In [ ]:
"""
Main Execution: Cohort-Level Feature Extraction

Executes complete radiomics extraction pipeline across all patients in
dataset. This is the primary analysis cell.

Execution Time Estimates (approximate):
    - Original mode: 5-10 minutes per 100 patients
    - Wavelet mode: 40-60 minutes per 100 patients
    - All mode: 2-3 hours per 100 patients

Output:
    - CSV file with complete feature matrix
    - Console logs for progress monitoring
    - Summary statistics and quality metrics
"""

In [89]:
print("="*70)
print("INITIATING BATCH RADIOMICS EXTRACTION")
print("="*70 + "\n")

df = process_multiple_patients(
    base_folder=base_folder,
    output_csv=OUTPUT_CSV,
    target_roi=TARGET_ROI,
    feature_mode=FEATURE_MODE
)

if df is not None:
    print(f"\n{'='*70}")
    print("PIPELINE EXECUTION COMPLETE")
    print(f"{'='*70}")
    print(f"Output file: {OUTPUT_CSV}")
    print(f"Patients processed: {len(df)}")
    print(f"Features per patient: {len(df.columns)}")
    print(f"Dataset shape: {df.shape}")
    print(f"{'='*70}\n")
else:
    print("\nBatch processing failed - review error logs above")


INITIATING BATCH RADIOMICS EXTRACTION


BATCH PROCESSING INITIATED
Total patients: 137
Target ROI: GTV-1
Feature mode: original
Output file: radiomics_features.csv


[1/137] 
Processing Patient: HN1004
Stage 1/4: Loading CT DICOM series...
  CT volume dimensions: (512, 512, 147)
  Voxel spacing: (0.9765625, 0.9765625, 3.0) mm
Stage 2/4: Locating ROI 'GTV-1'...
Found 1 RTSTRUCT file(s) to search
  Examining: 1-1.dcm
    Available structures: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'Brain', 'GTV-2', 'GTV-1']
    Status: Target structure 'GTV-1' located
Stage 3/4: Generating binary segmentation mask...
Converting structure to binary mask: GTV-1
  Raw mask dimensions: (512, 512, 147)
  Reference CT dimensions: (512, 512, 147)
  Transposed mask dimensions: (147, 512, 512)
  Final mask dimensions: (512, 512, 147)
  ROI voxel count: 42961
  ROI volume: 122.91 cc
  M

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  Feature extraction complete: 107 features computed
Status: Patient HN1004 processed successfully
  Total features extracted: 107

[2/137] 
Processing Patient: HN1006
Stage 1/4: Loading CT DICOM series...
  CT volume dimensions: (512, 512, 134)
  Voxel spacing: (0.9765625, 0.9765625, 3.0) mm
Stage 2/4: Locating ROI 'GTV-1'...
Found 1 RTSTRUCT file(s) to search
  Examining: 1-1.dcm
    Available structures: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'Cochlea-Right', 'Cochlea-Left', 'Brain', 'GTV-3', 'GTV-2', 'GTV-1']
    Status: Target structure 'GTV-1' located
Stage 3/4: Generating binary segmentation mask...
Converting structure to binary mask: GTV-1
  Raw mask dimensions: (512, 512, 134)
  Reference CT dimensions: (512, 512, 134)
  Transposed mask dimensions: (134, 512, 512)
  Final mask dimensions: (512, 512, 134)
  ROI voxel count: 1581
  ROI volume: 4.52 c

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  Feature extraction complete: 107 features computed
Status: Patient HN1006 processed successfully
  Total features extracted: 107

[3/137] 
Processing Patient: HN1022
Stage 1/4: Loading CT DICOM series...
  CT volume dimensions: (512, 512, 134)
  Voxel spacing: (0.9765625, 0.9765625, 3.0) mm
Stage 2/4: Locating ROI 'GTV-1'...
Found 1 RTSTRUCT file(s) to search
  Examining: 1-1.dcm
    Available structures: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'GTV-3', 'GTV-2', 'GTV-1']
    Status: Target structure 'GTV-1' located
Stage 3/4: Generating binary segmentation mask...
Converting structure to binary mask: GTV-1
  Raw mask dimensions: (512, 512, 134)
  Reference CT dimensions: (512, 512, 134)
  Transposed mask dimensions: (134, 512, 512)
  Final mask dimensions: (512, 512, 134)
  ROI voxel count: 12867
  ROI volume: 36.81 cc
  Mask generation successful
Stage 4/4

...

  Feature extraction complete: 107 features computed
Status: Patient HN1968 processed successfully
  Total features extracted: 107

[136/137] 
Processing Patient: HN1987
Stage 1/4: Loading CT DICOM series...
  CT volume dimensions: (512, 512, 134)
  Voxel spacing: (0.9765625, 0.9765625, 3.0) mm
Stage 2/4: Locating ROI 'GTV-1'...
Found 1 RTSTRUCT file(s) to search
  Examining: 1-1.dcm
    Available structures: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'Medulla-Oblongata', 'Cochlea-Right', 'Cochlea-Left', 'Brain', 'GTV-3', 'GTV-2', 'GTV-1']
    Status: Target structure 'GTV-1' located
Stage 3/4: Generating binary segmentation mask...
Converting structure to binary mask: GTV-1
  Raw mask dimensions: (512, 512, 134)
  Reference CT dimensions: (512, 512, 134)
  Transposed mask dimensions: (134, 512, 512)
  Final mask dimensions: (512, 512, 134)
  ROI voxel count: 28072
  ROI volume: 80.

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  Feature extraction complete: 107 features computed
Status: Patient HN1987 processed successfully
  Total features extracted: 107

[137/137] 
Processing Patient: HN1998
Stage 1/4: Loading CT DICOM series...
  CT volume dimensions: (512, 512, 307)
  Voxel spacing: (1.087890625, 1.087890625, 1.4999999999999738) mm
Stage 2/4: Locating ROI 'GTV-1'...
Found 1 RTSTRUCT file(s) to search
  Examining: 1-1.dcm
    Available structures: ['Spinal-Cord', 'Neck-Right', 'Neck-Left', 'Submandibular-Gland-Right', 'Submandibular-Gland-Left', 'Parotid-Right', 'Parotid-Left', 'Oral-Cavity', 'GTV-1']
    Status: Target structure 'GTV-1' located
Stage 3/4: Generating binary segmentation mask...
Converting structure to binary mask: GTV-1
  Raw mask dimensions: (512, 512, 307)
  Reference CT dimensions: (512, 512, 307)
  Transposed mask dimensions: (307, 512, 512)
  Final mask dimensions: (512, 512, 307)
  ROI voxel count: 3465
  ROI volume: 6.15 cc
  Mask generation successful
Stage 4/4: Extracting radiomi

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


  Feature extraction complete: 107 features computed
Status: Patient HN1998 processed successfully
  Total features extracted: 107

BATCH PROCESSING SUMMARY
Successfully processed: 137 patients (100.0%)
Failed processing: 0 patients
Features per patient: 107
Output file: radiomics_features.csv
File size: 0.26 MB

Quality Assessment:
  Status: Excellent (100.0% success rate)

PIPELINE EXECUTION COMPLETE
Output file: radiomics_features.csv
Patients processed: 137
Features per patient: 109
Dataset shape: (137, 109)



---

### **Cell 14: Results Visualization and Export Verification**

In [ ]:
"""
Results Inspection and Quality Control

Displays summary statistics and preview of extracted features for quality
verification before downstream analysis.

Checks:
    - Dataset dimensions (patients × features)
    - Feature completeness (no missing values)
    - Value ranges (sanity check for HU values)
    - Feature nomenclature verification
"""

In [91]:
if df is not None:
    print("="*70)
    print("RESULTS SUMMARY AND QUALITY CONTROL")
    print("="*70 + "\n")
    
    # Dataset dimensions
    print(f"Feature Matrix Dimensions: {df.shape}")
    print(f"  Patients (rows): {df.shape[0]}")
    print(f"  Features (columns): {df.shape[1]}")
    print(f"  Metadata columns: 2 (PatientID, ROI_Name)")
    print(f"  Radiomics features: {df.shape[1] - 2}")
    
    # Missing data check
    missing = df.isnull().sum().sum()
    print(f"\nData Completeness:")
    print(f"  Missing values: {missing}")
    if missing == 0:
        print(f"  Status: Complete (100% data coverage)")
    else:
        print(f"  Status: Incomplete - investigate missing data")
    
    # Display patient preview
    print("\n" + "="*70)
    print("PATIENT SAMPLE (First 5 Patients)")
    print("="*70)
    display(df.head())
    
    # Display feature nomenclature
    print("\n" + "="*70)
    print("FEATURE NOMENCLATURE (First 20 Features)")
    print("="*70)
    
    feature_cols = [col for col in df.columns if col not in ['PatientID', 'ROI_Name']]
    
    for i, col in enumerate(feature_cols[:20], 1):
        # Parse feature type
        if 'shape' in col.lower():
            feature_type = "[SHAPE]"
        elif 'firstorder' in col.lower():
            feature_type = "[FIRST-ORDER]"
        elif 'glcm' in col.lower():
            feature_type = "[GLCM TEXTURE]"
        elif 'glrlm' in col.lower():
            feature_type = "[GLRLM TEXTURE]"
        elif 'glszm' in col.lower():
            feature_type = "[GLSZM TEXTURE]"
        elif 'gldm' in col.lower():
            feature_type = "[GLDM TEXTURE]"
        elif 'ngtdm' in col.lower():
            feature_type = "[NGTDM TEXTURE]"
        else:
            feature_type = "[OTHER]"
        
        print(f"{i:3d}. {feature_type:20s} {col}")
    
    if len(feature_cols) > 20:
        print(f"\n... and {len(feature_cols) - 20} additional features")
    
    # Feature statistics preview
    print("\n" + "="*70)
    print("FEATURE STATISTICS PREVIEW")
    print("="*70)
    
    print("\n" + "="*70)
    print("EXPORT VERIFICATION COMPLETE")
    print("="*70)
    print(f"\nFeature matrix ready for downstream analysis:")
    print(f"  - Machine learning model development")
    print(f"  - Statistical analysis and hypothesis testing")
    print(f"  - Feature selection and dimensionality reduction")
    print(f"  - Clinical outcome correlation studies")
    print("\n" + "="*70)
else:
    print("No results to display - batch processing failed")

RESULTS SUMMARY AND QUALITY CONTROL

Feature Matrix Dimensions: (137, 109)
  Patients (rows): 137
  Features (columns): 109
  Metadata columns: 2 (PatientID, ROI_Name)
  Radiomics features: 107

Data Completeness:
  Missing values: 0
  Status: Complete (100% data coverage)

PATIENT SAMPLE (First 5 Patients)


,PatientID,ROI_Name,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,HN1004,GTV-1,0.759295,0.579260,43.402040,74.926725,87.2084652212731,87.2084652212731,72.90285978843376,92.06265546755797,...,1287.5957851586468,0.006091649950355723,7.704573394254903,0.15083447778217454,25382.1451931632,0.7386569363993765,0.00022174936904843307,8160.791193641188,0.0920944233175849,2.92934665418536
1,HN1006,GTV-1,0.696800,0.301773,11.687646,38.729977,29.058468512383033,38.42760289248472,34.897594502354764,41.98898970382184,...,1503.5185456526194,0.002805847957274179,6.649553739899192,0.22580645161290322,392.485794317727,0.04427043430246803,0.005246835265013618,3826.6394079165557,0.08127741751771043,21.14690716471923
2,HN1022,GTV-1,0.783281,0.560082,30.254456,54.017881,58.866938142305315,66.64742235263117,47.37083138919884,67.27194638560573,...,1366.0902545482081,0.0034646931386199662,7.280442050718658,0.14339006761482864,9401.43455350651,0.25935123060427023,0.0006516007393318773,4733.3261968478455,0.02992862289795048,4.86061361915349
3,HN1026,GTV-1,0.840406,0.744783,16.279669,21.858284,25.851043078039545,25.604232485888076,22.860742013124266,27.426260555650057,...,26.76499981426582,0.034304406157078166,4.925242942144948,0.0632996632996633,2879.5229741964686,1.5148683536866485,0.006854360363610231,36.20038449686023,0.07403125368824401,0.37204523859057326
4,HN1029,GTV-1,0.610856,0.319918,13.239173,41.382967,40.95692525767163,40.95692525767163,28.721564784868637,44.40895281733213,...,721.9929663802085,0.006384503538529918,6.967953904562411,0.2803406671398155,417.1315750680981,0.6303783413384101,0.0021675426310988686,7336.805704692472,0.4004345551616009,9.178849012368433



FEATURE NOMENCLATURE (First 20 Features)
  1. [SHAPE]              original_shape_Elongation
  2. [SHAPE]              original_shape_Flatness
  3. [SHAPE]              original_shape_LeastAxisLength
  4. [SHAPE]              original_shape_MajorAxisLength
  5. [SHAPE]              original_shape_Maximum2DDiameterColumn
  6. [SHAPE]              original_shape_Maximum2DDiameterRow
  7. [SHAPE]              original_shape_Maximum2DDiameterSlice
  8. [SHAPE]              original_shape_Maximum3DDiameter
  9. [SHAPE]              original_shape_MeshVolume
 10. [SHAPE]              original_shape_MinorAxisLength
 11. [SHAPE]              original_shape_Sphericity
 12. [SHAPE]              original_shape_SurfaceArea
 13. [SHAPE]              original_shape_SurfaceVolumeRatio
 14. [SHAPE]              original_shape_VoxelVolume
 15. [FIRST-ORDER]        original_firstorder_10Percentile
 16. [FIRST-ORDER]        original_firstorder_90Percentile
 17. [FIRST-ORDER]        original_firstorder_E

---